### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [2]:
# Dependencies and Setup
import os
import pandas as pd
import numpy as np

# File path
csvpath = os.path.join("Resources", "purchase_data.csv")

# Read Purchasing File and store into Pandas data frame
purchaseData_df = pd.read_csv(csvpath)
purchaseData_df.head()

,Purchase ID,SN,Age,Gender,Item ID,Item Name,Price
0,0,Lisim78,20,Male,108,"Extraction, Quickblade Of Trembling Hands",3.53
1,1,Lisovynya38,40,Male,143,Frenzied Scimitar,1.56
2,2,Ithergue48,24,Male,92,Final Critic,4.88
3,3,Chamassasya86,24,Male,100,Blindscythe,3.27
4,4,Iskosia90,23,Male,131,Fury,1.44


In [3]:
# Check for missing values
purchaseData_df.count()

Purchase ID    780
SN             780
Age            780
Gender         780
Item ID        780
Item Name      780
Price          780
dtype: int64

## Player Count

* Display the total number of players


In [4]:
# Display total number of players for the Player Count
totalPlayers = len(purchaseData_df["SN"].unique())
totalPlayers_df = pd.DataFrame({"Total Players": [totalPlayers]})
totalPlayers_df

,Total Players
0,576


## Purchasing Analysis (Total)

* Run basic calculations to obtain number of unique items, average price, etc.


* Create a summary data frame to hold the results


* Optional: give the displayed data cleaner formatting


* Display the summary data frame


In [6]:
# Purchasing Analysis (Total)
itemCount = len(purchaseData_df["Item ID"].unique())
totalRevenue = purchaseData_df["Price"].sum()
numPurchases = purchaseData_df["Price"].count()
meanItemPrice = purchaseData_df["Price"].mean()
# itemCount
# totalRevenue
# numPurchases
# meanItemPrice

pd.options.display.float_format = '${:,.2f}'.format
purchaseAnalysisTotal_df = pd.DataFrame({
    "Number of Unique Items": [itemCount], 
    "Average Price": [meanItemPrice], 
    "Number of Purhcases": [numPurchases], 
    "Total Revenue": [totalRevenue]
})

purchaseAnalysisTotal_df

,Number of Unique Items,Average Price,Number of Purhcases,Total Revenue
0,179,$3.05,780,"$2,379.77"


## Gender Demographics

* Percentage and Count of Male Players


* Percentage and Count of Female Players


* Percentage and Count of Other / Non-Disclosed




In [10]:
genderDemo = purchaseData_df[["SN", "Gender"]]
genderDemo.drop_duplicates(subset=["SN"])

,SN,Gender
0,Lisim78,Male
1,Lisovynya38,Male
2,Ithergue48,Male
3,Chamassasya86,Male
4,Iskosia90,Male
...,...,...
773,Hala31,Male
774,Jiskjask80,Male
775,Aethedru70,Female
777,Yathecal72,Male


In [17]:
purchCount = purchaseData_df.groupby(["Gender"]).count()["Item ID"]
purchCount

# avgPurch = purchaseData_df.groupby(["Gender"]).mean()["Price"]
# avgPurch

Gender
Female                   113
Male                     652
Other / Non-Disclosed     15
Name: Item ID, dtype: int64


## Purchasing Analysis (Gender)

* Run basic calculations to obtain purchase count, avg. purchase price, avg. purchase total per person etc. by gender




* Create a summary data frame to hold the results


* Optional: give the displayed data cleaner formatting


* Display the summary data frame

## Age Demographics

* Establish bins for ages


* Categorize the existing players using the age bins. Hint: use pd.cut()


* Calculate the numbers and percentages by age group


* Create a summary data frame to hold the results


* Optional: round the percentage column to two decimal points


* Display Age Demographics Table


## Purchasing Analysis (Age)

* Bin the purchase_data data frame by age


* Run basic calculations to obtain purchase count, avg. purchase price, avg. purchase total per person etc. in the table below


* Create a summary data frame to hold the results


* Optional: give the displayed data cleaner formatting


* Display the summary data frame

## Top Spenders

* Run basic calculations to obtain the results in the table below


* Create a summary data frame to hold the results


* Sort the total purchase value column in descending order


* Optional: give the displayed data cleaner formatting


* Display a preview of the summary data frame



## Most Popular Items

* Retrieve the Item ID, Item Name, and Item Price columns


* Group by Item ID and Item Name. Perform calculations to obtain purchase count, average item price, and total purchase value


* Create a summary data frame to hold the results


* Sort the purchase count column in descending order


* Optional: give the displayed data cleaner formatting


* Display a preview of the summary data frame



## Most Profitable Items

* Sort the above table by total purchase value in descending order


* Optional: give the displayed data cleaner formatting


* Display a preview of the data frame

